In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow.keras.backend as K
import pathlib
import os
import pickle
import glob
import numpy as np
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
from plotnine import *
import seaborn as sn
%matplotlib inline

import sys
sys.path.append("/tf/model/vggish")

import vggish_params 
import vggish_input
import mel_features

from kapre import STFT, Magnitude, MagnitudeToDecibel
from kapre.composed import get_melspectrogram_layer, get_log_frequency_spectrogram_layer

seed = 30
tf.random.set_seed(seed)
gpus = tf.config.list_physical_devices('GPU')

In [2]:
## need this if training on GPU
## tensorflow, get your shit together

if len(gpus)>0:

    from tensorflow.compat.v1 import ConfigProto
    from tensorflow.compat.v1 import InteractiveSession

    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)

In [3]:
filenames = list(glob.glob('/tf/data/ADReSS-IS2020/Full_wave_enhanced_audio/*/*'))
random.shuffle(filenames)

In [4]:
train_cutoff = int(len(filenames)*0.7)
val_cutoff = int(len(filenames)*0.85)

train_files = filenames[:train_cutoff]
val_files = filenames[train_cutoff:val_cutoff]
test_files = filenames[val_cutoff:]

In [5]:
meta_cc = pd.read_csv('/tf/data/ADReSS-IS2020/cc_meta_data.txt', sep=";", header=0, 
                  names = ['ID', 'Age', 'Gender', 'MMSE'])
meta_cd = pd.read_csv('/tf/data/ADReSS-IS2020/cd_meta_data.txt', sep=";", header=0, 
                      names = ['ID', 'Age', 'Gender', 'MMSE'])

meta = meta_cc.assign(Group = 'cc').append(meta_cd.assign(Group = 'cd')).reset_index()
meta.MMSE = pd.to_numeric(meta.MMSE.replace(' NA', 30))
meta.ID = meta.ID.str.strip()

In [6]:
def createDataset(files):
    
    allExamples = []
    allScores = []
    
    for filename in files:
        examples = vggish_input.wavfile_to_examples(filename)
        allExamples.append(examples)
        id = filename.split("/")[-1].split(".")[0]
        mmse = meta[meta["ID"]==id].iloc[0]["MMSE"]
        for i in range(examples.shape[0]):
            allScores.append(mmse)
    
    return tf.data.Dataset.from_tensor_slices((np.expand_dims(np.vstack(allExamples), axis=-1), np.array(allScores)))

train_dataset = createDataset(train_files)
val_dataset = createDataset(val_files)
test_dataset = createDataset(test_files)

In [7]:
for waveform, y in train_dataset.take(1):
    input_shape = waveform.shape
    print(input_shape)
    print(y)

(96, 64, 1)
tf.Tensor(26, shape=(), dtype=int64)


In [8]:
BATCH_SIZE = 256
SHUFFLE_BUFFER_SIZE = 100

train_dataset_b = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset_b = val_dataset.batch(BATCH_SIZE)
test_dataset_b = test_dataset.batch(BATCH_SIZE)

In [9]:
with open("vggish_model_weights.pkl", 'rb') as file:
    model_wts = pickle.load(file)

In [39]:
vggish = models.Sequential()

conv1  = layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv1',
                    input_shape=(vggish_params.NUM_FRAMES, vggish_params.NUM_BANDS, 1))

vggish.add(conv1)

# Set its weights, defined in model_vars
conv1.set_weights([model_wts['vggish/conv1/weights:0'], model_wts['vggish/conv1/biases:0']])
conv1.trainable = True # Freeze this layer

pool1  = layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pool1')
vggish.add(pool1)
# Note: no weights for pooling layers

conv2  = layers.Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv2')
vggish.add(conv2)
conv2.set_weights([model_wts['vggish/conv2/weights:0'], model_wts['vggish/conv2/biases:0']])
conv2.trainable = False

pool2  = layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pool2')
vggish.add(pool2)

conv3_1= layers.Conv2D(filters=256, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv3_1')
vggish.add(conv3_1)
conv3_1.set_weights([model_wts['vggish/conv3/conv3_1/weights:0'], model_wts['vggish/conv3/conv3_1/biases:0']])
conv3_1.trainable = False

conv3_2= layers.Conv2D(filters=256, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv3_2')
vggish.add(conv3_2)
conv3_2.set_weights([model_wts['vggish/conv3/conv3_2/weights:0'], model_wts['vggish/conv3/conv3_2/biases:0']])
conv3_2.trainable = False

pool3  = layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pool3')
vggish.add(pool3)

conv4_1= layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv4_1')
vggish.add(conv4_1)
conv4_1.set_weights([model_wts['vggish/conv4/conv4_1/weights:0'], model_wts['vggish/conv4/conv4_1/biases:0']])
conv4_1.trainable = False

conv4_2= layers.Conv2D(filters=512, kernel_size=(3,3), strides=1, padding='same', activation='relu', name='conv4_2')
vggish.add(conv4_2)
conv4_2.set_weights([model_wts['vggish/conv4/conv4_2/weights:0'], model_wts['vggish/conv4/conv4_2/biases:0']])
conv4_2.trainable = False

pool4  = layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pool4')
vggish.add(pool4)

vggish.add(layers.Flatten())

fc1_1  = layers.Dense(4096, activation='relu', name='fc1_1')
vggish.add(fc1_1)
fc1_1.set_weights([model_wts['vggish/fc1/fc1_1/weights:0'], model_wts['vggish/fc1/fc1_1/biases:0']])
fc1_1.trainable = False

fc1_2  = layers.Dense(4096, activation='relu', name='fc1_2')
vggish.add(fc1_2)
fc1_2.set_weights([model_wts['vggish/fc1/fc1_2/weights:0'], model_wts['vggish/fc1/fc1_2/biases:0']])
fc1_2.trainable = False

# Note: first trainable layer in network
fc2 = layers.Dense(vggish_params.EMBEDDING_SIZE, activation='relu', name='fc2')
vggish.add(fc2)
fc2.set_weights([model_wts['vggish/fc2/weights:0'], model_wts['vggish/fc2/biases:0']])
fc2.trainable = True

vggish.add(layers.Dropout(0.25))

vggish.add(layers.Dense(128, activation='relu'))

vggish.add(layers.Dropout(0.25))

vggish.add(layers.Dense(64, activation='relu'))

vggish.add(layers.Dropout(0.25))

vggish.add(layers.Dense(64, activation='relu'))

vggish.add(layers.Dropout(0.25))

vggish.add(layers.Dense(1, activation='relu'))

vggish.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 96, 64, 64)        640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 32, 128)       73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 24, 16, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 24, 16, 256)       295168    
_________________________________________________________________
conv3_2 (Conv2D)             (None, 24, 16, 256)       590080    
_________________________________________________________________
pool3 (MaxPooling2D)         (None, 12, 8, 256)       

In [40]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)

vggish.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0),
    loss='mse',
    metrics='mse'
)

In [ ]:
EPOCHS = 10000
history = vggish.fit(
    train_dataset_b, 
    validation_data=val_dataset_b,  
    epochs=EPOCHS,
    callbacks=[callback]
)

Epoch 1/10000
23/23 [==============================] - 3s 99ms/step - loss: 560.1972 - mse: 560.1972 - val_loss: 612.6764 - val_mse: 612.6764
Epoch 2/10000
23/23 [==============================] - 2s 95ms/step - loss: 542.9898 - mse: 542.9898 - val_loss: 581.3358 - val_mse: 581.3358
Epoch 3/10000
23/23 [==============================] - 2s 95ms/step - loss: 503.3196 - mse: 503.3196 - val_loss: 497.5323 - val_mse: 497.5323
Epoch 4/10000
23/23 [==============================] - 2s 95ms/step - loss: 408.7952 - mse: 408.7952 - val_loss: 361.4200 - val_mse: 361.4200
Epoch 5/10000
23/23 [==============================] - 2s 96ms/step - loss: 277.8491 - mse: 277.8491 - val_loss: 213.1275 - val_mse: 213.1275
Epoch 6/10000
23/23 [==============================] - 2s 97ms/step - loss: 178.4147 - mse: 178.4147 - val_loss: 146.6124 - val_mse: 146.6124
Epoch 7/10000
23/23 [==============================] - 2s 97ms/step - loss: 143.2698 - mse: 143.2698 - val_loss: 106.0596 - val_mse: 106.0596
Epoch 

Epoch 118/10000
23/23 [==============================] - 2s 96ms/step - loss: 60.7021 - mse: 60.7021 - val_loss: 66.6555 - val_mse: 66.6555
Epoch 119/10000
23/23 [==============================] - 2s 98ms/step - loss: 59.3353 - mse: 59.3353 - val_loss: 56.6978 - val_mse: 56.6978
Epoch 120/10000
 3/23 [==>...........................] - ETA: 1s - loss: 58.1916 - mse: 58.1916

In [ ]:
metrics = history.history
plt.plot(history.epoch, np.sqrt(metrics['mse']), np.sqrt(metrics['val_mse']))
plt.legend(['rmse', 'val_rmse'])
plt.show()

In [ ]:
np.sqrt(vggish.evaluate(train_dataset.batch(BATCH_SIZE)))

In [ ]:
np.sqrt(vggish.evaluate(test_dataset.batch(BATCH_SIZE)))